<a href="https://colab.research.google.com/github/kouchan0708/q-macro/blob/main/project2%E4%BF%AE%E6%AD%A3%E7%89%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
from tabulate import tabulate
from google.colab import files

# ファイルをアップロード
uploaded = files.upload()

# アップロードしたファイルの名前を取得
filename = list(uploaded.keys())[0]

# アップロードしたStataファイルを読み込む
pwt_data = pd.read_stata(filename)

# 必要な列のみを抽出する
columns_needed = ['country', 'year', 'rgdpe', 'pop', 'emp', 'avh', 'labsh', 'rtfpna', 'ctfp', 'cn']
pwt = pwt_data[columns_needed]

# 資本分配率を計算
pwt['alpha'] = 1 - pwt['labsh']

# 各年度における労働者一人あたりの労働生産性を計算(実質GDP/労働従事人口)
pwt['labor_productivity'] = pwt['rgdpe'] / pwt['emp']

# 労働生産性の成長率の計算
pwt['lp_growth_rate'] = pwt.groupby('country')['labor_productivity'].pct_change() * 100

# 技術進歩の再計算を一人当たりの指標に変更
pwt['tfp_growth_per_pop'] = (pwt['rtfpna'] / pwt['emp']).pct_change() * 100

# 資本深化の再計算を一人当たりの指標に変更（αはそれぞれの国で計算した値を用いている）
pwt['capital_deepening_per_pop'] = pwt.groupby('country').apply(lambda x: x['alpha'] * (x['cn'] / x['emp']).pct_change() * 100).reset_index(level=0, drop=True)

# 必要なデータを抽出
growth_data = pwt[['country', 'year', 'lp_growth_rate', 'tfp_growth_per_pop', 'capital_deepening_per_pop']].dropna()

# 1990年から2019年までのデータを抽出
filtered_growth_data = growth_data[(growth_data['year'] >= 1990) & (growth_data['year'] <= 2019)]

# OECD加盟国リスト
oecd_countries = [
    'Australia', 'Austria', 'Belgium', 'Canada', 'Denmark', 'Finland', 'France', 'Germany',
    'Greece', 'Iceland', 'Ireland', 'Italy', 'Japan', 'Netherlands', 'New Zealand', 'Norway',
    'Portugal', 'Spain', 'Sweden', 'Switzerland', 'United Kingdom', 'United States'
]

# OECD加盟国のデータを抽出
oecd_data = filtered_growth_data[filtered_growth_data['country'].isin(oecd_countries)]

# 各国の平均値を計算
oecd_data_avg = oecd_data.groupby('country').mean().reset_index()

# TFP Shareの再計算：それぞれの国においてTFP Growthの平均を労働生産性の成長率の平均で割る
oecd_data_avg['tfp_share'] = oecd_data_avg['tfp_growth_per_pop'] / oecd_data_avg['lp_growth_rate']

# Capital Shareの再計算：それぞれの国において資本深化の平均を労働生産性の成長率の平均で割る
oecd_data_avg['capital_share'] = oecd_data_avg['capital_deepening_per_pop'] / oecd_data_avg['lp_growth_rate']

# 全体の平均値を計算
overall_avg = oecd_data_avg.mean(numeric_only=True)
overall_avg['country'] = 'Average'

# 平均値の行をデータフレームに追加
overall_avg_df = pd.DataFrame(overall_avg).transpose()
oecd_data_avg = pd.concat([oecd_data_avg, overall_avg_df], ignore_index=True)

# 表の作成
table_data = oecd_data_avg[['country', 'lp_growth_rate', 'tfp_growth_per_pop', 'capital_deepening_per_pop', 'tfp_share', 'capital_share']].copy()
table_data.columns = ['Country', 'Growth Rate (%)', 'TFP Growth  (%)', 'Capital Deepening (%)', 'TFP Share', 'Capital Share']

# 表の表示（線で区切る）
print(tabulate(table_data, headers='keys', tablefmt='grid', showindex=False))


Saving pwt1001.dta to pwt1001 (3).dta


<ipython-input-4-4896a97f4680>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pwt['alpha'] = 1 - pwt['labsh']
<ipython-input-4-4896a97f4680>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pwt['labor_productivity'] = pwt['rgdpe'] / pwt['emp']
<ipython-input-4-4896a97f4680>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

+----------------+-------------------+-------------------+-------------------------+-------------+-----------------+
| Country        |   Growth Rate (%) |   TFP Growth  (%) |   Capital Deepening (%) |   TFP Share |   Capital Share |
+================+===================+===================+=========================+=============+=================+
| Australia      |           1.48909 |        -1.17772   |                0.756288 | -0.790904   |        0.507888 |
+----------------+-------------------+-------------------+-------------------------+-------------+-----------------+
| Austria        |           2.30983 |        -0.556364  |                1.18565  | -0.240867   |        0.513304 |
+----------------+-------------------+-------------------+-------------------------+-------------+-----------------+
| Belgium        |           1.89358 |        -0.870823  |                1.23518  | -0.459882   |        0.652296 |
+----------------+-------------------+-------------------+------

<ipython-input-4-4896a97f4680>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pwt['capital_deepening_per_pop'] = pwt.groupby('country').apply(lambda x: x['alpha'] * (x['cn'] / x['emp']).pct_change() * 100).reset_index(level=0, drop=True)
